# EDA & SHAP/LIME Explainability Analysis for AQI Prediction

## Overview
This notebook performs comprehensive Exploratory Data Analysis (EDA) and uses SHAP and LIME for model-agnostic feature importance explanations.

In [2]:
!pip install seaborn

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

import sys
sys.path.insert(0, '../')

print("Libraries loaded successfully!")

ModuleNotFoundError: No module named 'seaborn'

## Section 1: Load Data and Models

In [ ]:
from src.mongodb_handler import MongoDBHandler
from src.feature_engineering import FeatureEngineer
import joblib
import json

# Load data from MongoDB
print("Loading data from MongoDB...")
db_handler = MongoDBHandler()
df = db_handler.get_training_data(days=60)
db_handler.close()

print(f"[OK] Loaded {len(df)} records")
print(f"\nData shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# Load trained models
print("Loading trained models...")
models = {}
model_dir = '../models/saved_models'

models['Ridge'] = joblib.load(f'{model_dir}/ridge_latest.pkl')
models['Gradient Boosting'] = joblib.load(f'{model_dir}/gradient_boosting_latest.pkl')
models['Random Forest'] = joblib.load(f'{model_dir}/random_forest_latest.pkl')

print("[OK] All models loaded")
print(f"\nModels: {list(models.keys())}")

In [ ]:
# Load feature names
with open('../models/saved_models/feature_names.json', 'r') as f:
    feature_data = json.load(f)
    if isinstance(feature_data, dict) and 'features' in feature_data:
        feature_names = feature_data['features']
    else:
        feature_names = feature_data

print(f"Number of features: {len(feature_names)}")
print(f"\nFeatures: {feature_names[:10]}...")

## Section 2: Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("Data Statistics:")
print("="*60)
print(f"\nAQI - Target Variable:")
print(df['aqi'].describe())

print(f"\n\nPollutant Concentrations:")
pollutants = ['pm25', 'pm10', 'no2', 'so2', 'co', 'o3']
print(df[pollutants].describe())

In [ ]:
# AQI Distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram
axes[0, 0].hist(df['aqi'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_title('AQI Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('AQI Value')
axes[0, 0].set_ylabel('Frequency')

# Box plot
axes[0, 1].boxplot(df['aqi'], vert=True)
axes[0, 1].set_title('AQI Box Plot', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('AQI Value')

# Time series
if 'timestamp' in df.columns:
    df_sorted = df.sort_values('timestamp')
    axes[1, 0].plot(df_sorted['timestamp'], df_sorted['aqi'], color='steelblue', alpha=0.7)
    axes[1, 0].set_title('AQI Time Series', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Date')
    axes[1, 0].set_ylabel('AQI Value')
    axes[1, 0].tick_params(axis='x', rotation=45)

# Pollutants correlation with AQI
pollutants_with_aqi = pollutants + ['aqi']
corr_with_aqi = df[pollutants_with_aqi].corr()['aqi'].drop('aqi').sort_values(ascending=False)
axes[1, 1].barh(corr_with_aqi.index, corr_with_aqi.values, color='coral')
axes[1, 1].set_title('Pollutant Correlation with AQI', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Correlation Coefficient')

plt.tight_layout()
plt.savefig('../notebooks/plots/eda_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nPollutant Correlations with AQI:")
print(corr_with_aqi)

In [ ]:
# Correlation matrix heatmap
fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = df[pollutants_with_aqi].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax, cbar_kws={'label': 'Correlation'})
ax.set_title('Correlation Matrix: Pollutants and AQI', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../notebooks/plots/correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Section 3: Prepare Features and Generate Predictions

In [ ]:
# Prepare features using the same approach as model training
print("Preparing features...")

# Use the same column exclusion as ModelTrainer
exclude_cols = ['aqi', 'timestamp', 'date', '_id', 'inserted_at', 
                'station_name', 'latitude', 'longitude']

# Use features that the model was trained on (from feature_names.json)
# Only use columns that exist in both feature_names and df
available_features = [f for f in feature_names if f in df.columns]

if not available_features:
    print(f"[WARN] No matching features. Falling back to numeric columns...")
    available_features = [col for col in df.columns 
                         if col not in exclude_cols 
                         and df[col].dtype in ['int64', 'float64']]

print(f"Using {len(available_features)} features: {available_features[:5]}...")

# Create feature matrix directly from DataFrame columns
X = df[available_features].copy()
X = X.fillna(X.median(numeric_only=True))
y = df['aqi'].values

print(f"[OK] Feature matrix shape: {X.shape}")
print(f"[OK] Target shape: {y.shape}")

# Update feature_names to match what we're using
feature_names = available_features

## Section 4: SHAP Feature Importance Analysis

In [ ]:
import shap

print("Generating SHAP Explanations...\n")

# For each model
shap_results = {}

for model_name, model in models.items():
    print(f"Processing {model_name}...")
    
    try:
        # Sample data for SHAP
        background_data = X.iloc[:min(100, len(X))]
        
        # Create SHAP explainer
        if 'RandomForest' in str(type(model).__name__) or 'GradientBoosting' in str(type(model).__name__):
            explainer = shap.TreeExplainer(model)
        else:
            explainer = shap.KernelExplainer(model.predict, background_data)
        
        # Calculate SHAP values
        shap_values = explainer.shap_values(X.iloc[:min(100, len(X))])
        
        # Handle different formats
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        # Calculate feature importance
        feature_importance = np.abs(shap_values).mean(axis=0)
        importance_df = pd.DataFrame({
            'Feature': X.columns,
            'Importance': feature_importance
        }).sort_values('Importance', ascending=False)
        
        shap_results[model_name] = importance_df
        
        print(f"[OK] {model_name} - Top 10 Features:")
        for idx, row in importance_df.head(10).iterrows():
            print(f"    {row['Feature']:.<40} {row['Importance']:.4f}")
        print()
        
    except Exception as e:
        print(f"[FAIL] Error with {model_name}: {e}\n")

In [ ]:
# Visualize SHAP importance for all models
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for idx, (model_name, importance_df) in enumerate(shap_results.items()):
    top_features = importance_df.head(15)
    axes[idx].barh(top_features['Feature'], top_features['Importance'], color=f'C{idx}')
    axes[idx].set_xlabel('Mean |SHAP value|')
    axes[idx].set_title(f'SHAP Feature Importance - {model_name}', fontweight='bold')
    axes[idx].invert_yaxis()

plt.suptitle('SHAP Feature Importance Across All Models', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../notebooks/plots/shap_importance_all_models.png', dpi=300, bbox_inches='tight')
plt.show()

print("SHAP importance plots saved!")

## Section 5: LIME Feature Importance Analysis

In [ ]:
import lime
import lime.lime_tabular

print("Generating LIME Explanations...\n")

lime_results = {}

for model_name, model in models.items():
    print(f"Processing {model_name}...")
    
    try:
        # Create LIME explainer
        explainer = lime.lime_tabular.LimeTabularExplainer(
            X.values,
            feature_names=X.columns.tolist(),
            mode='regression',
            verbose=False
        )
        
        # Explain multiple instances
        lime_importances = pd.DataFrame(columns=['Feature', 'Weight'])
        
        for i in range(min(5, len(X))):
            exp = explainer.explain_instance(X.iloc[i], model.predict, num_features=20)
            exp_list = exp.as_list()
            
            for feature, weight in exp_list:
                feature_name = feature.split()[0] if feature else 'unknown'
                if feature_name in X.columns:
                    lime_importances = pd.concat([
                        lime_importances,
                        pd.DataFrame({'Feature': [feature_name], 'Weight': [abs(weight)]})
                    ], ignore_index=True)
        
        # Aggregate
        if len(lime_importances) > 0:
            lime_agg = lime_importances.groupby('Feature')['Weight'].mean().sort_values(ascending=False)
            lime_results[model_name] = lime_agg
            
            print(f"[OK] {model_name} - Top 10 Features:")
            for idx, (feature, weight) in enumerate(lime_agg.head(10).items()):
                print(f"    {feature:.<40} {weight:.4f}")
            print()
        else:
            print(f"[FAIL] No LIME importances for {model_name}\n")
            
    except Exception as e:
        print(f"[FAIL] Error with {model_name}: {e}\n")

In [ ]:
# Visualize LIME importance
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

for idx, (model_name, importance_series) in enumerate(lime_results.items()):
    top_features = importance_series.head(15)
    axes[idx].barh(top_features.index, top_features.values, color=f'C{idx+3}')
    axes[idx].set_xlabel('Mean Weight')
    axes[idx].set_title(f'LIME Feature Importance - {model_name}', fontweight='bold')
    axes[idx].invert_yaxis()

plt.suptitle('LIME Feature Importance Across All Models', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../notebooks/plots/lime_importance_all_models.png', dpi=300, bbox_inches='tight')
plt.show()

print("LIME importance plots saved!")

## Section 6: Key Findings Summary

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS - FEATURE IMPORTANCE SUMMARY")
print("="*80)

print("\n1. SHAP ANALYSIS (SHAPLEY VALUES):")
print("-" * 80)
for model_name, importance_df in shap_results.items():
    print(f"\n{model_name}:")
    print(f"  Top 5 Features:")
    for idx, row in importance_df.head(5).iterrows():
        print(f"    â€¢ {row['Feature']:.<40} {row['Importance']:.4f}")

print("\n\n2. LIME ANALYSIS (LOCAL INTERPRETABLE MODEL-AGNOSTIC):")
print("-" * 80)
for model_name, importance_series in lime_results.items():
    print(f"\n{model_name}:")
    print(f"  Top 5 Features:")
    for idx, (feature, weight) in enumerate(importance_series.head(5).items()):
        print(f"    â€¢ {feature:.<40} {weight:.4f}")

print("\n\n3. TREND ANALYSIS:")
print("-" * 80)
print(f"  Data Points: {len(df)}")
print(f"  AQI Range: {df['aqi'].min():.1f} - {df['aqi'].max():.1f}")
print(f"  AQI Mean: {df['aqi'].mean():.1f}")
print(f"  AQI Std Dev: {df['aqi'].std():.1f}")

print("\n  Top Pollutant Correlations with AQI:")
corr_with_aqi_sorted = df[pollutants_with_aqi].corr()['aqi'].drop('aqi').sort_values(ascending=False)
for feature, corr in corr_with_aqi_sorted.head(5).items():
    print(f"    â€¢ {feature:.<40} {corr:.4f}")

print("\n" + "="*80)
print("Analysis Complete!")
print("="*80)